# This is a simple instruction on how to apply the feasibility predictor. The main steps include:<br>
* Load the feasibility predictor
* Parse a packing instance
* Pass the feature of the instance to the predictor and obtain the prediction

In [119]:
import torch as T
import pickle as pkl
import numpy as np

## Step 1. Load the feasibility predictor

In [120]:
predictor = T.jit.load('Feasibility_Predictor.pt')
device = T.device("cuda" if T.cuda.is_available() else "cpu")
predictor  = predictor.to(device)

## Step2. Parse a packing instance

In [121]:
def extract_manual_feature(items, bin_width, bin_height):
    """
    The method is to extract features from a set of items
    Parameters:
    ----------
    items (a list of np.arrays)
    bin_width (int): the width of a bin
    bin_height (int): the height of a bin
    Returns (a list of metrics)
    -------
    notes:
    we extract five types of features:
    1) the ratio between width and height, and four statistical metrics (mean, min, max, std) as the features
    2) the ratio between width and the bin width, four statistical metrics (mean, min, max, std) as the features
    3) the ratio between height and the bin height, four statistical metrics (mean, min, max, std) as the features
    4) the ratio between area of a item and the bin capacity, four statistical metrics (mean, min, max, std) as the features
    5) the ratio between total area of the items and the bin capacity, a single metric
    """
    MAX_W_H_RATIO = 18
    capacity = bin_width * bin_height
    w_h_ratios = np.asarray(list(map(lambda x: x[0] / x[1], items))) / MAX_W_H_RATIO
    w_bin_ratios = np.asarray(list(map(lambda x: x[0] / bin_width, items)))
    h_bin_ratios = np.asarray(list(map(lambda x: x[1] / bin_height, items)))
    area_capacity_ratios = np.asarray(list(map(lambda x: (x[0] * x[1]) / capacity, items)))
    total_area = np.asarray(list(map(lambda x: x[0] * x[1], items))).sum()
    w_h_features = [w_h_ratios.mean(), w_h_ratios.min(), w_h_ratios.max(), w_h_ratios.std()]
    w_bin_features = [w_bin_ratios.mean(), w_bin_ratios.min(), w_bin_ratios.max(), w_bin_ratios.std()]
    h_bin_features = [h_bin_ratios.mean(), h_bin_ratios.min(), h_bin_ratios.max(), h_bin_ratios.std()]
    area_capacity_features = [area_capacity_ratios.mean(), area_capacity_ratios.min(),
                              area_capacity_ratios.max(), area_capacity_ratios.std()]
    total_area_capacity_features = [total_area / capacity]
    extracted_features = [w_h_features, w_bin_features, h_bin_features, area_capacity_features,
                          total_area_capacity_features]
    result = []
    for x in extracted_features:
        result.extend(x)
    return result

In [122]:
# Demo instance A, result: infeasible 
# width, height
# 6,4
# 17,1
# 8,1
# 15,2
# 2,6
# 13,1
# 4,7
# 7,4
# 6,4
# 2,5

# Demo instance B, result: feasible:
# 2,9
# 3,4
# 2,6
# 13,1
# 4,7
# 7,4
# 6,4
# 2,5
instance = [
           # width, height
            [2,9], 
            [3,4],
            [2,6],
            [13,1],
            [4,7],
            [7,4],
            [6,4],
            [2,5]
            ]
features = extract_manual_feature(instance, bin_width = 20, bin_height = 10)

## Step3. Pass the feature of the instance to the predictor and obtain the prediction

In [123]:
predictor.eval()

RecursiveScriptModule(
  original_name=FFNNFeasibilityChecker
  (linear1): RecursiveScriptModule(original_name=Linear)
  (linear2): RecursiveScriptModule(original_name=Linear)
  (linear3): RecursiveScriptModule(original_name=Linear)
)

In [124]:
features = T.tensor(features, dtype=T.float).to(device)
features

tensor([0.1287, 0.0123, 0.7222, 0.2262, 0.2438, 0.1000, 0.6500, 0.1775, 0.5000,
        0.1000, 0.9000, 0.2236, 0.0906, 0.0500, 0.1400, 0.0352, 0.7250],
       device='cuda:0')

In [125]:
with T.no_grad():
    y = predictor(features)
y
if y > 0.5:
    print("infeasible")
else:
    print("feasible")

feasible
